# Trabalho

dados: https://raw.githubusercontent.com/cibelerusso/Datasets/refs/heads/main/DadoseDecisoes.csv

modelo: regressor logistico para o sexo de funcionarios de uma determinada empresa se baseando em seus dados trabalhistas, como: salario, posição, tempo de serviço etc. a ideia é entender se uma determinada empresa é equalitaria ou não. se o modelo conseguir captar o a variancia dos dados em relação ao sexo dos funcionário, então a empresa não é equalitaria.

# Libs

In [ ]:
install.packages('fastDummies')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
install.packages('caret')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘sparsevctrs’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’




# Dados

In [ ]:
data <- read.csv('https://raw.githubusercontent.com/cibelerusso/Datasets/refs/heads/main/DadoseDecisoes.csv')
head(data)

,ID,Idade,Gênero,Departamento,Salário,Horas_Trabalhadas,Produtividade,Satisfação,Tempo_Empresa,Cursos_Realizados,Home_Office
,<int>,<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<chr>
1,1,34,Masculino,RH,3848.89,36,84.89,94.33,0,4,Não
2,2,39,Feminino,Vendas,2536.32,48,107.84,98.62,4,9,Sim
3,3,39,Masculino,Marketing,7377.81,44,95.66,80.23,11,6,Sim
4,4,27,Feminino,Vendas,2536.32,40,88.89,94.63,8,1,Sim
5,5,41,Feminino,Vendas,2536.32,37,80.97,86.44,11,12,Não
6,6,39,Feminino,Vendas,2536.32,36,81.80,73.90,2,4,Não


# EDA

In [ ]:
summary(data)

       ID             Idade          Gênero          Departamento      
 Min.   :  1.00   Min.   :18.00   Length:230         Length:230        
 1st Qu.: 58.25   1st Qu.:24.00   Class :character   Class :character  
 Median :115.50   Median :29.00   Mode  :character   Mode  :character  
 Mean   :115.50   Mean   :30.55                                        
 3rd Qu.:172.75   3rd Qu.:35.00                                        
 Max.   :230.00   Max.   :54.00                                        
    Salário      Horas_Trabalhadas Produtividade      Satisfação    
 Min.   : 2536   Min.   :30.00     Min.   : 65.74   Min.   : 39.42  
 1st Qu.: 2536   1st Qu.:34.00     1st Qu.: 77.12   1st Qu.: 77.57  
 Median : 5668   Median :38.00     Median : 87.04   Median : 87.73  
 Mean   : 8030   Mean   :38.85     Mean   : 87.85   Mean   : 86.15  
 3rd Qu.:12129   3rd Qu.:44.00     3rd Qu.: 98.42   3rd Qu.: 96.19  
 Max.   :29053   Max.   :48.00     Max.   :112.06   Max.   :100.00  
 Tempo_Empres

In [ ]:
# drop id
data <- data[, -which(names(data) == "ID")]

In [ ]:
# encode das variáveis categóricas
encoded <- fastDummies::dummy_cols(data, select_columns = c('Gênero', 'Departamento', 'Home_Office'),
                                       remove_first_dummy = TRUE,
                                       remove_selected_columns = TRUE)

In [ ]:
# novos dados
head(encoded)

,Idade,Salário,Horas_Trabalhadas,Produtividade,Satisfação,Tempo_Empresa,Cursos_Realizados,Gênero_Masculino,Departamento_RH,Departamento_TI,Departamento_Vendas,Home_Office_Sim
,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,34,3848.89,36,84.89,94.33,0,4,1,1,0,0,0
2,39,2536.32,48,107.84,98.62,4,9,0,0,0,1,1
3,39,7377.81,44,95.66,80.23,11,6,1,0,0,0,1
4,27,2536.32,40,88.89,94.63,8,1,0,0,0,1,1
5,41,2536.32,37,80.97,86.44,11,12,0,0,0,1,0
6,39,2536.32,36,81.80,73.90,2,4,0,0,0,1,0


In [ ]:
attach(encoded)

# Treino/Teste

In [ ]:
indice <- caret::createDataPartition(
  Gênero_Masculino,
  p = 0.8,
  list = F)

In [ ]:
treino <- encoded[indice, ]
teste  <- encoded[-indice, ]

In [ ]:
modelo <- glm(
  Gênero_Masculino ~ . - Gênero_Masculino,
  data = treino,
  family = binomial(link = "logit")
)

summary(modelo)


Call:
glm(formula = Gênero_Masculino ~ . - Gênero_Masculino, family = binomial(link = "logit"), 
    data = treino)

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)          1.561e+00  2.856e+00   0.547   0.5846    
Idade               -3.775e-02  3.010e-02  -1.254   0.2097    
Salário              1.794e-04  4.372e-05   4.104 4.06e-05 ***
Horas_Trabalhadas    7.228e-02  1.117e-01   0.647   0.5177    
Produtividade       -3.566e-02  5.591e-02  -0.638   0.5237    
Satisfação          -6.316e-03  1.825e-02  -0.346   0.7293    
Tempo_Empresa       -7.739e-02  4.514e-02  -1.715   0.0864 .  
Cursos_Realizados    1.167e-02  5.327e-02   0.219   0.8266    
Departamento_RH     -1.099e-01  5.180e-01  -0.212   0.8320    
Departamento_TI     -8.815e-01  4.782e-01  -1.843   0.0653 .  
Departamento_Vendas  6.264e-01  4.906e-01   1.277   0.2017    
Home_Office_Sim     -3.855e-01  6.112e-01  -0.631   0.5282    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0

In [ ]:
stepwise_aic <- function(x, y, verbose = TRUE) {

    # regressoras, sem a target
    dados <- subset(x, select = -c(y))

    # distribuição da target
    formula_nula <- as.formula("y ~ 1")             # inicia a formula com uma regressora
    formula_completa <- as.formula(paste(           # vai adicionando demais regressores à formula
        "y ~", paste(names(x),collapse = " + ")
    ))

    # steps
    modelo_step <- step(glm(formula_nula, data = dados, family = binomial),
        scope = list(lower = formula_nula, upper = formula_completa),
        direction = "both", trace = ifelse(verbose, 1, 0))

    # resultados
    if (verbose) {
        cat("\nModelo final:\n")
        print(summary (modelo_step))
    }

    return (modelo_step)
}

In [ ]:
x <- subset(treino, select = -c(Gênero_Masculino))
y <- `Gênero_Masculino`

modelo_final <- stepwise_aic(x, y)
modelo_final

ERROR: Error in model.frame.default(formula = y ~ Idade + Salário + Horas_Trabalhadas + : variable lengths differ (found for 'Salário')
